## Bull Flag Detector

### Prepare data

Source: polygon.io

#### Get data from source

In [40]:
import requests
from datetime import datetime, timedelta

API_KEY = "6tGnu5OaAcMVxe6mweYxHQtFQVuAaGfu"
symbol = "C:EURUSD"
multiplier = 4
timespan = "hour"
to_date = datetime.utcnow()
from_date = to_date - timedelta(days=68)

url = (
    f"https://api.polygon.io/v2/aggs/ticker/{symbol}/range/"
    f"{multiplier}/{timespan}/{from_date.date()}/{to_date.date()}?adjusted=true&sort=asc&limit=5000&apiKey={API_KEY}"
)

response = requests.get(url)
data = response.json()

print(url)

https://api.polygon.io/v2/aggs/ticker/C:EURUSD/range/4/hour/2025-08-07/2025-10-14?adjusted=true&sort=asc&limit=5000&apiKey=6tGnu5OaAcMVxe6mweYxHQtFQVuAaGfu


In [ ]:
import requests
from datetime import datetime, timedelta
import pandas as pd
import time

API_KEY = "6tGnu5OaAcMVxe6mweYxHQtFQVuAaGfu"
symbol = "C:EURUSD"
multiplier = 15
timespan = "minute"
to_date = datetime.utcnow()
from_date = to_date - timedelta(days=1500)

url = (
    f"https://api.polygon.io/v2/aggs/ticker/{symbol}/range/"
    f"{multiplier}/{timespan}/{from_date.date()}/{to_date.date()}?adjusted=true&sort=asc&limit=1000000&apiKey={API_KEY}"
)

all_results = []
results_valid = False
error = ""
# "status":"OK" "status":"ERROR" "status":"DELAYED"
#"error":"You've exceeded the maximum requests per minute, please wait or upgrade your subscription to continue. https://polygon.io/pricing"}

print(url)

while url:
    # parse message
    r = requests.get(url)
    data = r.json()

    # validate message
    if (not "status" in data) or (not "results" in data and not "error" in data):
        results_valid = False
        error = f"Invalid response: {data}"
        break
    if data["status"] == "ERROR":
        # handle timeout
        if "exceeded the" in data["error"]:
            time.sleep(15)
            continue
        # handle other
        results_valid = False
        error = data["error"]
        break
    
    # collect results
    all_results.extend(data["results"])


    # handle paging
    if "next_url" in data:
        url = data.get("next_url")
        url += f"&apiKey={API_KEY}"
        print(url)
        continue

    if data["status"] == "OK":
        break

    print(data)
    break
    raise f"{data}"

if not results_valid:
    # request failed
    print(f"{error}")
    pass

# request successful
print(f"Retrieved {len(all_results)} total bars")

# convert to DataFrame
df = pd.DataFrame(all_results)
df["timestamp"] = pd.to_datetime(df["t"], unit="ms").dt.strftime('%Y-%m-%d %H:%M')
df = df.rename(columns={"o":"open","h":"high","l":"low","c":"close"})
df = df[["timestamp","open","high","low","close"]]
print(df.head())

filename = f"../data/{symbol.replace('C:', '')}_{multiplier}{timespan}_{from_date.date()}_{to_date.date()}.csv"
import os
os.makedirs(os.path.dirname(filename), exist_ok=True)

df.to_csv(filename, index=False)
print(f"Saved {len(df)} rows to {filename}")


https://api.polygon.io/v2/aggs/ticker/C:EURUSD/range/15/minute/2024-06-02/2025-10-15?adjusted=true&sort=asc&limit=1000000&apiKey=6tGnu5OaAcMVxe6mweYxHQtFQVuAaGfu
https://api.polygon.io/v2/aggs/ticker/C:EURUSD/range/15/minute/1721605500000/2025-10-15?cursor=bGltaXQ9NTAwMDAmc29ydD1hc2M&apiKey=6tGnu5OaAcMVxe6mweYxHQtFQVuAaGfu
https://api.polygon.io/v2/aggs/ticker/C:EURUSD/range/15/minute/1725647400000/2025-10-15?cursor=bGltaXQ9NTAwMDAmc29ydD1hc2M&apiKey=6tGnu5OaAcMVxe6mweYxHQtFQVuAaGfu
https://api.polygon.io/v2/aggs/ticker/C:EURUSD/range/15/minute/1730083500000/2025-10-15?cursor=bGltaXQ9NTAwMDAmc29ydD1hc2M&apiKey=6tGnu5OaAcMVxe6mweYxHQtFQVuAaGfu
https://api.polygon.io/v2/aggs/ticker/C:EURUSD/range/15/minute/1734381000000/2025-10-15?cursor=bGltaXQ9NTAwMDAmc29ydD1hc2M&apiKey=6tGnu5OaAcMVxe6mweYxHQtFQVuAaGfu
https://api.polygon.io/v2/aggs/ticker/C:EURUSD/range/15/minute/1738824300000/2025-10-15?cursor=bGltaXQ9NTAwMDAmc29ydD1hc2M&apiKey=6tGnu5OaAcMVxe6mweYxHQtFQVuAaGfu
https://api.polygon.io/

next, save data as csv

```text
timestamp,open,high,low,close
1672531200000,1.0704,1.0705,1.0703,1.0704
...
```

In [39]:
import csv

if "results" not in data:
    print("Error:", data)
    exit()

filename = f"../data/{symbol.replace('C:', '')}_{multiplier}{timespan}_{from_date.date()}_{to_date.date()}.csv"

with open(filename, "w", newline="") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["timestamp", "open", "high", "low", "close"])

    for bar in data["results"]:
        ts = datetime.utcfromtimestamp(bar["t"] / 1000).strftime("%Y-%m-%d %H:%M")
        writer.writerow([ts, bar["o"], bar["h"], bar["l"], bar["c"]])

print(f"Saved {len(data['results'])} records to {filename}")


Saved 21 records to ../data/EURUSD_4hour_2025-08-07_2025-10-14.csv


Run Labelstudio with the command:

```bash
docker run -it -p 8080:8080 -v "D:\BME\Deep Learning\Bull Flag Detector\data:/label-studio/data" heartexlabs/label-studio:latest
```

Labeling interface:

```xml
<View>
    <Header>Válasszon egy címkét, majd jelölje ki a 'zászló' szakaszt az idősoron!</Header>
    <TimeSeriesLabels name="label" toName="ts">
        <Label value="Bullish Normal" background="#228B22"/>
        <Label value="Bullish Wedge" background="#006400"/>
        <Label value="Bullish Pennant" background="#32CD32"/>
        <Label value="Bearish Normal" background="#DC143C"/>
        <Label value="Bearish Wedge" background="#8B0000"/>
        <Label value="Bearish Pennant" background="#FF4500"/>
    </TimeSeriesLabels>
    <TimeSeries name="ts" valueType="url" value="$csv" sep=","
        timeColumn="timestamp"
        timeFormat="%Y-%m-%d %H:%M"
        timeDisplayFormat="%Y-%m-%d"
        overviewChannels="close">
        <Channel column="close" displayFormat=",.1f" strokeColor="#00FF00" legend="Close Price"/>
    </TimeSeries>
</View>
```